<div style="font-size:18pt; padding-top:20px; text-align:center">СЕМИНАР. <b>Регуляризация</b></div><hr>
<div style="text-align:right;">Папулин С.Ю. <span style="font-style: italic;font-weight: bold;">(papulin.study@mail.ru)</span></div>

<a name="0"></a>
<div><span style="font-size:14pt; font-weight:bold">Содержание</span>
    <ol>
        <li><a href="#1">Регуляризация</a></li>
        <li><a href="#2">Кросс-валидация</a>
        <li><a href="#3">Регуляризация и кросс-валидация</a></li>
        <li><a href="#4">Выбор модели</a></li>
        <li><a href="#5">Источники</a>
        </li>
    </ol>
</div>

Подключение библиотек

In [ ]:
from sklearn import datasets
from scipy import stats
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn import linear_model
import matplotlib.pyplot as plt

%matplotlib inline

<a name="1"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:16pt; font-weight:bold">1. Линейная полиномиальная регрессия</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

In [ ]:
from sklearn.linear_model import Ridge, Lasso, RidgeCV
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from numpy.polynomial.polynomial import polyval

In [ ]:
def regression_dataset(n=100):
    """Генерация исходных данных"""
    x = stats.uniform.rvs(size=n, loc=0, scale=6, random_state=0)
    f = lambda x: np.sin(x)
    y = stats.norm.rvs(size=n, loc=0, scale=0.5, random_state=0) + f(x)
    return (x, y, f)


# Инициализация исходных данных
x, y, f = regression_dataset()

# График
xx = np.linspace(0, 6, 100)
plt.plot(x, y, "o", color="green", label="observed data")
plt.plot(xx, f(xx), "-", color="SteelBlue", label="true function")
plt.title("Initial data")
plt.grid(True)
plt.xlabel("X")
plt.ylabel("Y")
plt.legend()
plt.show()

In [ ]:
# Параметры
REGULARIZATION = 0
POLY_DEGREE = 15

In [ ]:
# Представление признака x в матричной форме
X_ = x.reshape(-1, 1)

# Формирование обучающего и тестового подмножеств
X_train, X_test, y_train, y_test = train_test_split(X_, y, test_size=0.3, random_state=200)

# Формирование последовательности действий
pipeline = Pipeline([
    ("transformation", PolynomialFeatures(degree=POLY_DEGREE, include_bias=False)), 
    ("linear_model", Ridge(alpha=REGULARIZATION, fit_intercept=True, normalize=True))
])

# Обучение
pipeline.fit(X_train, y_train)

# Параметры обученной модели
print("Параметры модели:")
print("\tw{} = {}".format(0, pipeline.named_steps["linear_model"].intercept_))
for indx, coef in enumerate(pipeline.named_steps["linear_model"].coef_):
    print("\tw{} = {}".format(indx+1, coef))

In [ ]:
# Предсказания на обучающем и тестовом множествах
y_train__pred = pipeline.predict(X_train)
y_test__pred = pipeline.predict(X_test)

# Ошибки на обучающем множестве
mse_traint = mean_squared_error(y_train, y_train__pred)
r2_train = r2_score(y_train, y_train__pred)

print("Обучающее множество:")
print("\tTrain MSE:", mse_traint)
print("\tTrain R^2:", r2_train)

# Ошибки на тестовом множестве
mse_test = mean_squared_error(y_test, y_test__pred)
r2_test = r2_score(y_test, y_test__pred)

print("\nТестовое множество:")
print("\tTest MSE:", mse_test)
print("\tTest R^2:", r2_test)

In [ ]:
# Графики
xx = np.linspace(0, 6, 100).reshape(-1, 1)
plt.figure(figsize=[12, 4])

plt.subplot(1,2,1)

plt.title("Train data")
plt.plot(X_train, y_train, "o", color="green")
plt.plot(xx, pipeline.predict(xx), color="red", lw=2, label="predicted")
plt.scatter(X_train, y_train__pred, color="red")
plt.vlines(X_train, ymin=y_train, ymax=y_train__pred, colors="black", linestyles="dotted")
plt.plot(xx, f(xx), "-", color="SteelBlue", label="true function")
plt.xlabel("X")
plt.ylabel("Y")
plt.legend()
plt.grid(True)


plt.subplot(1,2,2)

plt.title("Test data")
plt.plot(X_test, y_test, "o", color="green")
plt.plot(xx, pipeline.predict(xx), color="red", lw=2, label="predicted")
plt.scatter(X_test, y_test__pred, color="red")
plt.vlines(X_test, ymin=y_test, ymax=y_test__pred, colors="black", linestyles="dotted")
plt.plot(xx, f(xx), "-", color="SteelBlue", label="true function")
plt.xlabel("X")
plt.ylabel("Y")
plt.legend()
plt.grid(True)

plt.tight_layout()

plt.show()

<a name="2"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:16pt; font-weight:bold">2. Регуляризация в линейной регрессии</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

In [ ]:
# Параметры
REGULARIZATION = 0.01
POLY_DEGREE = 15

# Формирование последовательности действий с ридж-регрессией
pipeline = Pipeline([
    ("transformation", PolynomialFeatures(degree=POLY_DEGREE, include_bias=False)), 
    ("linear_model", Ridge(alpha=REGULARIZATION, fit_intercept=True, normalize=True))
])

# Формирование последовательности действий с лассо-регрессией
# pipeline = Pipeline([
#     ("transformation", PolynomialFeatures(degree=15)), 
#     ("linear_model", Lasso(alpha=REGULARIZATION, fit_intercept=False))
# ])

# Обучение
pipeline.fit(X_train, y_train)

# Параметры обученной модели
print("Параметры модели:")
print("\tw{} = {}".format(0, pipeline.named_steps["linear_model"].intercept_))
for indx, coef in enumerate(pipeline.named_steps["linear_model"].coef_):
    print("\tw{} = {}".format(indx+1, coef))

In [ ]:
# Предсказания на обучающем и тестовом множествах
y_train__pred = pipeline.predict(X_train)
y_test__pred = pipeline.predict(X_test)

# Ошибки на обучающем множестве
mse_traint = mean_squared_error(y_train, y_train__pred)
r2_train = r2_score(y_train, y_train__pred)

print("Обучающее множество:")
print("\tTrain MSE:", mse_traint)
print("\tTrain R^2:", r2_train)

# Ошибки на тестовом множестве
mse_test = mean_squared_error(y_test, y_test__pred)
r2_test = r2_score(y_test, y_test__pred)

print("\nТестовое множество:")
print("\tTest MSE:", mse_test)
print("\tTest R^2:", r2_test)

In [ ]:
# Графики
xx = np.linspace(0, 6, 100).reshape(-1, 1)
plt.figure(figsize=[12, 4])

plt.subplot(1,2,1)

plt.title("Train data")
plt.plot(X_train, y_train, "o", color="green")
plt.plot(xx, pipeline.predict(xx), color="red", lw=2, label="predicted")
plt.scatter(X_train, y_train__pred, color="red")
plt.vlines(X_train, ymin=y_train, ymax=y_train__pred, colors="black", linestyles="dotted")
plt.plot(xx, f(xx), "-", color="SteelBlue", label="true function")
plt.xlabel("X")
plt.ylabel("Y")
plt.legend()
plt.grid(True)


plt.subplot(1,2,2)

plt.title("Test data")
plt.plot(X_test, y_test, "o", color="green")
plt.plot(xx, pipeline.predict(xx), color="red", lw=2, label="predicted")
plt.scatter(X_test, y_test__pred, color="red")
plt.vlines(X_test, ymin=y_test, ymax=y_test__pred, colors="black", linestyles="dotted")
plt.plot(xx, f(xx), "-", color="SteelBlue", label="true function")
plt.xlabel("X")
plt.ylabel("Y")
plt.legend()
plt.grid(True)

plt.tight_layout()

plt.show()

<a name="3"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:16pt; font-weight:bold">3. Регуляризация с кросс-валидацией</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

### Выбор модели

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
kf = KFold(n_splits=5, random_state=0)

In [ ]:
alphas = [0.0000001, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100]

RidgeCV

In [ ]:
# Формирование последовательности действий с ридж-регрессией
pipeline = Pipeline([
    ("transformation", PolynomialFeatures(degree=POLY_DEGREE, include_bias=False)), 
    ("linear_model", RidgeCV(alphas=alphas, fit_intercept=True, normalize=True, cv=kf, scoring="neg_mean_squared_error"))
])

# Обучение
pipeline.fit(X_train, y_train)

# Параметры обученной модели
print("Параметры модели:")
print("\tw{} = {}".format(0, pipeline.named_steps["linear_model"].intercept_))
for indx, coef in enumerate(pipeline.named_steps["linear_model"].coef_):
    print("\tw{} = {}".format(indx+1, coef))

In [ ]:
# Предсказания на обучающем и тестовом множествах
y_train__pred = pipeline.predict(X_train)
y_test__pred = pipeline.predict(X_test)

# Ошибки на обучающем множестве
mse_traint = mean_squared_error(y_train, y_train__pred)
r2_train = r2_score(y_train, y_train__pred)

print("Обучающее множество:")
print("\tTrain MSE:", mse_traint)
print("\tTrain R^2:", r2_train)

# Ошибки на тестовом множестве
mse_test = mean_squared_error(y_test, y_test__pred)
r2_test = r2_score(y_test, y_test__pred)

print("\nТестовое множество:")
print("\tTest MSE:", mse_test)
print("\tTest R^2:", r2_test)

In [ ]:
pipeline.named_steps["linear_model"].alpha_

GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# Формирование последовательности действий с ридж-регрессией
pipeline = Pipeline([
    ("transformation", PolynomialFeatures(degree=POLY_DEGREE, include_bias=False)), 
    ("linear_model", Ridge(alpha=REGULARIZATION, fit_intercept=True, normalize=True))
])

# Сетка параметров
parameters = {
    "linear_model__alpha": alphas
}

# Параметры обучения
grid_class_parameters = {
    "estimator": pipeline,
    "param_grid": parameters,
    "cv": kf,
    "scoring": "neg_mean_squared_error",
    "return_train_score": True
}


# Обучение
# grid_search = GridSearchCV(estimator=pipeline, param_grid=parameters, cv=kf, return_train_score=True, scoring="neg_mean_squared_error")
grid_search = GridSearchCV(**grid_class_parameters)
grid_search.fit(X_train, y_train)

# Параметры обученной модели
print("Параметры модели:")
print("\tw{} = {}".format(0, grid_search.best_estimator_.named_steps["linear_model"].intercept_))
for indx, coef in enumerate(grid_search.best_estimator_.named_steps["linear_model"].coef_):
    print("\tw{} = {}".format(indx+1, coef))

In [ ]:
# Предсказания на обучающем и тестовом множествах
y_train__pred = grid_search.predict(X_train)
y_test__pred = grid_search.predict(X_test)

# Ошибки на обучающем множестве
mse_traint = mean_squared_error(y_train, y_train__pred)
r2_train = r2_score(y_train, y_train__pred)

print("Обучающее множество:")
print("\tTrain MSE:", mse_traint)
print("\tTrain R^2:", r2_train)

# Ошибки на тестовом множестве
mse_test = mean_squared_error(y_test, y_test__pred)
r2_test = r2_score(y_test, y_test__pred)

print("\nТестовое множество:")
print("\tTest MSE:", mse_test)
print("\tTest R^2:", r2_test)

In [ ]:
# Вывод сведений по обучению
grid_search.cv_results_

In [ ]:
# График

log_alphas = np.log10(np.asarray(grid_search.cv_results_["param_linear_model__alpha"], dtype="float"))
log_alphas

plt.figure(figsize=[6, 4])

plt.subplot(1,1,1)
plt.title("Model Selection")
plt.plot(log_alphas, -grid_search.cv_results_["mean_test_score"], "o-", label="Validation")
plt.plot(log_alphas, -grid_search.cv_results_["mean_train_score"], "o-", label="Train")
plt.xlabel("$log_{10}(Regularization)$")
plt.ylabel("$MSE$")
plt.legend()
plt.grid(True)

plt.show()

In [ ]:
# Графики
xx = np.linspace(0, 6, 100).reshape(-1, 1)
plt.figure(figsize=[12, 4])

plt.subplot(1,2,1)

plt.title("Train data")
plt.plot(X_train, y_train, "o", color="green")
plt.plot(xx, grid_search.predict(xx), color="red", lw=2, label="predicted")
plt.scatter(X_train, y_train__pred, color="red")
plt.vlines(X_train, ymin=y_train, ymax=y_train__pred, colors="black", linestyles="dotted")
plt.plot(xx, f(xx), "-", color="SteelBlue", label="true function")
plt.xlabel("X")
plt.ylabel("Y")
plt.legend()
plt.grid(True)


plt.subplot(1,2,2)

plt.title("Test data")
plt.plot(X_test, y_test, "o", color="green")
plt.plot(xx, grid_search.predict(xx), color="red", lw=2, label="predicted")
plt.scatter(X_test, y_test__pred, color="red")
plt.vlines(X_test, ymin=y_test, ymax=y_test__pred, colors="black", linestyles="dotted")
plt.plot(xx, f(xx), "-", color="SteelBlue", label="true function")
plt.xlabel("X")
plt.ylabel("Y")
plt.legend()
plt.grid(True)

plt.tight_layout()

plt.show()

<a name="4"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:16pt; font-weight:bold">4.Задание</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

In [ ]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, RidgeClassifier

In [ ]:
digits = datasets.load_digits()
print(digits.DESCR)

In [ ]:
IMAGE_INDX = 3

print("Image matrix:\n", digits["images"][IMAGE_INDX])
print("Target value:", digits.target[IMAGE_INDX])

plt.imshow(digits.images[IMAGE_INDX])
plt.show()

In [ ]:
# Параметры регуляризации
Cs = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]

# Способ разделения данных
kfolds = KFold(n_splits=5, shuffle=True, random_state=0)

# Исходные данные для обучения
X = digits["images"].reshape(len(digits["images"]), -1)
y = digits["target"]

# Формирование обучающего и тестового подмножеств
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=200)

In [ ]:
# Параметры инициализации модели
model_class_parameters = {
    "penalty": "l2", 
    "Cs": Cs,
    "cv": kfolds, 
    "multi_class": "ovr", 
    "solver": "newton-cg", 
    "max_iter": 200,
    "random_state": 12345
}

# Обучение
logistic_model = LogisticRegressionCV(**model_class_parameters)
logistic_model.fit(X_train, y_train)

In [ ]:
# Параметры обученной модели
for i in range(10):
    print("Параметры модели {}:".format(i+1))
    print("\tw{} = {}".format(0, logistic_model.intercept_[i]))
    for indx, coef in enumerate(logistic_model.coef_[i].flatten()):
        print("\tw{} = {}".format(indx+1, coef))

In [ ]:
# Оценка качества модел
train_error = logistic_model.score(X_train, y_train)
test_error = logistic_model.score(X_test, y_test)

print("Train Accuracy:", train_error)
print("Test Accuracy:", test_error)

In [ ]:
# Сетка параметров
parameters = {
    "C": Cs
}

# Параметры модели
model_class_parameters = {
    "penalty": "l2", 
    "C": float("inf"),
    "multi_class": "ovr", 
    "solver": "newton-cg", 
    "max_iter": 200,
    "random_state": 12345
}

logistic_model = LogisticRegression(**model_class_parameters)

# Параметры сетки
grid_class_parameters = {
    "estimator": logistic_model, 
    "param_grid": parameters,
    "cv": kfolds,
    "return_train_score": True
}

# Обучение
grid_search = GridSearchCV(**grid_class_parameters)
grid_search.fit(X_train, y_train)

In [ ]:
# Оценка качества модел
train_error = grid_search.score(X_train, y_train)
test_error = grid_search.score(X_test, y_test)

print("Train Accuracy:", train_error)
print("Test Accuracy:", test_error)

In [ ]:
# График

log_Cs = np.log10(np.asarray(Cs, dtype="float"))
log_Cs

plt.figure(figsize=[6, 4])

plt.subplot(1,1,1)
plt.title("Model Selection")
plt.plot(log_Cs, grid_search.cv_results_["mean_test_score"], "o-", label="Validation")
plt.plot(log_Cs, grid_search.cv_results_["mean_train_score"], "o-", label="Train")
plt.xlabel("$log_{10}(Regularization)$")
plt.ylabel("$MSE$")
plt.legend()
plt.grid(True)

plt.show()

[1.1. Linear Models](https://scikit-learn.org/stable/modules/linear_model.html)